<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/05_Create_Vector_DB/Create_Vector_Database_FAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Vector Database

In [1]:
#installs
!pip install tqdm
!pip install -q sentence_transformers
!pip install -q pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 4.8 MB/s eta 0:00:00


In [2]:
#imports
from tqdm import tqdm
import os
from google.colab import userdata
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from pinecone import Pinecone, PodSpec, ServerlessSpec
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Import Files From GitHub

In [3]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 1913, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 1913 (delta 186), reused 275 (delta 160), pack-reused 1595
Receiving objects: 100% (1913/1913), 359.53 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (1269/1269), done.
Updating files: 100% (1121/1121), done.


In [4]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
text_df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN
...,...,...,...,...,...,...,...
10121,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10122,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10123,Earnings Call Q&A,EQ-340,EOG,May-03-2019,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10124,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [5]:
url =  '/content/Financial_Sentiment_LLM/03_Sentiment_Analysis/Sentiment_Analysis_Results.csv'
sentiment_df = pd.read_csv(url)
sentiment_df

,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves / Exploration / Acquisitions / Mergers / Divestments,Environment / Regulatory / Geopolitics,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price
0,Investment Research,IR-1,MRO,5/16/2024,NaN,Positive,Neutral,Positive,Negative,Neutral,Neutral
1,Investment Research,IR-2,EOG,5/14/2024,NaN,Positive,Neutral,Positive,Neutral,Positive,Neutral
2,Investment Research,IR-3,EOG,5/11/2024,NaN,Positive,Positive,Positive,Neutral,Neutral,Neutral
3,Investment Research,IR-4,DVN,5/11/2024,NaN,Positive,Neutral,Positive,Negative,Neutral,Neutral
4,Investment Research,IR-5,COP,5/7/2024,NaN,Positive,Neutral,Neutral,Negative,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...
10120,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,NaN,Positive,Neutral,Negative,Positive,Positive,Neutral
10121,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,NaN,Positive,Positive,Neutral,Positive,Positive,Neutral
10122,Earnings Call Q&A,EQ-340,EOG,May-03-2019,NaN,Positive,Positive,Positive,Neutral,Neutral,Neutral
10123,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,NaN,Positive,Positive,Neutral,Neutral,Positive,Neutral


In [6]:
# Merge text and sentiment dataframes
sentiment_df_subset = sentiment_df[["Unique_ID", "Alternative Energy / Lower Carbon",
                                    "Oil Price / Natural Gas Price / Gasoline Price",
                                    "Environment / Regulatory / Geopolitics",
                                    "Production"]]

# Drop NANs or articles where sentiment could not be determined
sentiment_df_subset = sentiment_df_subset.dropna()

# Merge the DataFrames on 'Unique_ID'
df_text_sentiment = pd.merge(text_df, sentiment_df_subset, on="Unique_ID", how='right')
df_text_sentiment

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price,Environment / Regulatory / Geopolitics,Production
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,Positive,Neutral,Neutral,Neutral
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,Neutral,Neutral,Neutral,Positive
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,Neutral,Neutral,Negative,Neutral
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,Neutral,Neutral,Negative,Neutral
...,...,...,...,...,...,...,...,...,...,...,...
10698,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Neutral
10699,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Positive
10700,Earnings Call Q&A,EQ-340,EOG,May-03-2019,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Positive
10701,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Neutral,Positive


In [7]:
def chunk_text(text):
    max_word_count = 500
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_word_count = 0

    for sentence in sentences:
        word_count = len(sentence.split())
        if current_word_count + word_count <= max_word_count:
            current_chunk.append(sentence)
            current_word_count += word_count
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_word_count = word_count

    # Add the last chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

#Write chunks to dataframe
tqdm.pandas()
df_text_sentiment['Text Chunks'] = df_text_sentiment['Article Text'].progress_apply(chunk_text)
df_text_sentiment

100%|██████████| 10703/10703 [00:48<00:00, 219.39it/s]


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price,Environment / Regulatory / Geopolitics,Production,Text Chunks
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"[Stock Report | May 16, 2024 | NYSESymbol: MRO..."
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,Positive,Neutral,Neutral,Neutral,"[Stock Report | May 14, 2024 | NYSESymbol: EOG..."
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,Neutral,Neutral,Neutral,Positive,"[Stock Report | May 11, 2024 | NYSESymbol: EOG..."
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,Neutral,Neutral,Negative,Neutral,"[Stock Report | May 11, 2024 | NYSESymbol: DVN..."
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,Neutral,Neutral,Negative,Neutral,"[Stock Report | May 07, 2024 | NYSESymbol: COP..."
...,...,...,...,...,...,...,...,...,...,...,...,...
10698,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Neutral,[Question and Answer\nOperator\n[Operator Inst...
10699,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Positive,[Question and Answer\nOperator\n[Operator Inst...
10700,Earnings Call Q&A,EQ-340,EOG,May-03-2019,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Positive,[Question and Answer\nOperator\n[Operator Inst...
10701,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Neutral,Positive,[Question and Answer\nOperator\n[Operator Inst...


In [8]:
#split into chunks and create new dataframe
df_chunk = df_text_sentiment.explode('Text Chunks').reset_index(drop=True)
df_chunk

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price,Environment / Regulatory / Geopolitics,Production,Text Chunks
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"Stock Report | May 16, 2024 | NYSESymbol: MRO ..."
1,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,2024 0.55 E 0.70 E 0.77 E 0.80 E 2.82\nproduct...
2,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"2024 EBITDA, in line with MRO’s historical\nav..."
3,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"Chairman, President & CEOSenior VP of Technolo..."
4,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"Collectively, from 2017 through 2019,\n1,681\n..."
...,...,...,...,...,...,...,...,...,...,...,...,...
82679,Earnings Call Q&A,EQ-342,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,"We also,\nduring the quarter, completed the sa..."
82680,Earnings Call Q&A,EQ-342,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,William L. Bullock\nPresident of Asia Pacific ...
82681,Earnings Call Q&A,EQ-342,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,Michael Anthony Hall\nHeikkinen Energy Advisor...
82682,Earnings Call Q&A,EQ-342,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,"And\nthat's one of the reasons, along with wha..."


### Encode Chunks

In [9]:
#create text embeddings of chunks
embedding_model = SentenceTransformer("all-mpnet-base-v2")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model.to(device)
text_embeddings = embedding_model.encode(df_chunk['Text Chunks'].tolist())
text_embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(82684, 768)

### Create Vector Database

In [11]:
#create vector
vectors = []
for idx in range(text_embeddings.shape[0]):
  vector = [float(i) for i in list(text_embeddings[idx])]
  vectors.append({"id": str(df_chunk['Unique_ID'][idx]),
                  "metadata": {"date": str(df_chunk['Date'][idx]),
                               "alt_energy": df_chunk['Alternative Energy / Lower Carbon'][idx],
                               "oil_price": df_chunk['Oil Price / Natural Gas Price / Gasoline Price'][idx],
                               "env_reg": df_chunk['Environment / Regulatory / Geopolitics'][idx],
                               "prod": df_chunk['Production'][idx],
                               "ticker": df_chunk['Ticker'][idx]},
                  "values": vector })

#vectors[text_embeddings.shape[0]-1]

In [12]:
#delete existing index to rebuild with new data
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

#pc.delete_index("newsembedding2")

In [14]:
#create vector database index
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

index_name = "fastvectors"

pc.create_index(
    name=index_name,
    dimension=text_embeddings.shape[1], # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [15]:
# insert vectors into database
index = pc.Index(index_name)

# segments set to batch into vector database because of 2MB send limit
segments = 20
sections = int(text_embeddings.shape[0] / segments)
for i in tqdm (range(sections+1), desc="Loading..."):
  index.upsert(
    vectors=vectors[segments*i:segments*(1+i)],
    namespace="ns1"
  )

Loading...: 100%|██████████| 4135/4135 [14:50<00:00,  4.64it/s]


In [16]:
#check index
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 9905}},
 'total_vector_count': 9905}

In [ ]:
#from google.colab import runtime
#runtime.unassign()